DS methods: 
1. supervised
   - classification
     - metrical methods: kNN (with advanced cores)
     - methods powered by trees:randomForest, boosting; 
     - linear methods
   - regression
2. unsupervised
   - clasterization
   - reduce dimension

Dataset: size `l X d` (X - features, y - target)

DS method 'create' function from dataset.

In [43]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [44]:
data = pd.read_csv('../../data/telecom_churn.csv')

In [45]:
data.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [46]:
data.drop(['State', 'Voice mail plan'], axis=1, inplace=True)

In [47]:
data['International plan'] = data['International plan'].map({'Yes': 1, 'No': 0})

In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Account length          3333 non-null   int64  
 1   Area code               3333 non-null   int64  
 2   International plan      3333 non-null   int64  
 3   Number vmail messages   3333 non-null   int64  
 4   Total day minutes       3333 non-null   float64
 5   Total day calls         3333 non-null   int64  
 6   Total day charge        3333 non-null   float64
 7   Total eve minutes       3333 non-null   float64
 8   Total eve calls         3333 non-null   int64  
 9   Total eve charge        3333 non-null   float64
 10  Total night minutes     3333 non-null   float64
 11  Total night calls       3333 non-null   int64  
 12  Total night charge      3333 non-null   float64
 13  Total intl minutes      3333 non-null   float64
 14  Total intl calls        3333 non-null   

In [49]:
y = data['Churn'].astype('int')

In [50]:
X = data.drop('Churn', axis=1)

In [51]:
X.shape, y.shape

((3333, 17), (3333,))

In [52]:
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np

In [53]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=17)

In [54]:
X_train.shape, X_test.shape

((2333, 17), (1000, 17))

In [55]:
first_tree = DecisionTreeClassifier(random_state=17)

In [56]:
np.mean(cross_val_score(first_tree, X_train, y_train, cv=5))

0.9138423504976518

In [57]:
from sklearn.neighbors import KNeighborsClassifier

In [58]:
first_knn = KNeighborsClassifier()

In [59]:
np.mean(cross_val_score(first_knn, X_train, y_train, cv=5))

0.8671274043984523

### set max_depth for Tree

In [60]:
from sklearn.model_selection import GridSearchCV

In [61]:
tree_params = {'max_depth': np.arange(1, 11), 'max_features': [.5, .7, 1.]}

In [62]:
tree_grid = GridSearchCV(first_tree, tree_params, cv=5, n_jobs=-1)

In [63]:
%%time
tree_grid.fit(X_train, y_train)

Wall time: 3.31 s


GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=17,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 1

In [64]:
tree_grid.best_score_, tree_grid.best_params_

(0.9417108564391468, {'max_depth': 6, 'max_features': 1.0})

In [65]:
knn_params = {'n_neighbors': [1, 2, 3, 4] + list(range(50, 100, 10))}

In [66]:
knn_grid = GridSearchCV(first_knn, knn_params, cv=5)

In [67]:
%%time
knn_grid.fit(X_train, y_train)

Wall time: 1.55 s


GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 3, 4, 50, 60, 70, 80, 90]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [68]:
knn_grid.best_score_, knn_grid.best_params_

(0.8658416887998456, {'n_neighbors': 4})

In [69]:
# value 4 on border of list, could be check >4
knn_params = {'n_neighbors': list(range(5, 30, 5))}
knn_grid = GridSearchCV(first_knn, knn_params, cv=5)

In [70]:
%%time
knn_grid.fit(X_train, y_train)
knn_grid.best_score_, knn_grid.best_params_

Wall time: 754 ms


(0.8701289391697531, {'n_neighbors': 10})

In [71]:
tree_grid.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features=1.0, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=17, splitter='best')

In [73]:
tree_valid_pred = tree_grid.predict(X_valid)

In [74]:
from sklearn.metrics import accuracy_score

In [75]:
accuracy_score(y_valid, tree_valid_pred)

0.946

In [78]:
# check result with 'baseline' from dataset
1 - np.mean(y)

0.8550855085508551

In [79]:
from sklearn.tree import export_graphviz

In [81]:
export_graphviz(tree_grid.best_estimator_, out_file='telecom_tree.dot',
                feature_names=X.columns, filled=True, rounded=True)

In [93]:
!ls -l *.dot

-rw-r--r-- 1 vmj 197121 8137 May 15 14:04 telecom_tree.dot
-rw-r--r-- 1 vmj 197121 1815 May 15 14:12 telecom_tree2.dot


In [83]:
!dot -Tpng telecom_tree.dot -o telecom_tree.png

<img src='telecom_tree.png'>

In [94]:
second_tree = DecisionTreeClassifier(max_depth=3).fit(X_train, y_train)
second_tree.score(X_valid, y_valid)

0.905

In [97]:
export_graphviz(second_tree, out_file='telecom_tree3.dot',
                feature_names=X.columns, filled=True, rounded=True)

In [98]:
!dot -Tpng telecom_tree3.dot -o telecom_tree3.png
!ls -l *.dot

-rw-r--r-- 1 vmj 197121 8137 May 15 14:04 telecom_tree.dot
-rw-r--r-- 1 vmj 197121 1819 May 15 14:15 telecom_tree2.dot
-rw-r--r-- 1 vmj 197121 1819 May 15 14:16 telecom_tree3.dot


<img src='telecom_tree3.png'>

categorical features can be convert to numbers. One-hot-encoding (get_dummies) - binary features (0/1)